In [11]:
import pandas as pd
from datetime import datetime
from typing import Optional

## Data Processing and Cleaning

In [4]:
df = pd.read_csv('../data/my_activity_data=20241206180702.csv')
print('Dataframe Shape:', df.shape)
df.head()

Dataframe Shape: (339, 60)


,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id,...,suffer_score,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state,average_cadence,average_temp,max_watts,weighted_average_watts
0,2,volume velo,19283.1,5234,5496,226.0,Ride,Ride,NaN,12974413480,...,24.0,64383208,1,a12974413480,iaciHmceNp@lAfArC`BxCXv@rAzB\~@rClFz@tB`AvApAl...,2,NaN,NaN,NaN,NaN
1,2,Entraînement aux poids de nuit,0.0,3743,3743,0.0,WeightTraining,WeightTraining,NaN,12939605259,...,7.0,64383208,1,a12939605259,NaN,2,NaN,NaN,NaN,NaN
2,2,Entraînement aux poids de nuit,0.0,1811,1811,0.0,WeightTraining,WeightTraining,NaN,12932144110,...,4.0,64383208,1,a12932144110,NaN,2,NaN,NaN,NaN,NaN
3,2,Volume,9440.7,3779,3782,57.0,Run,Run,NaN,12924227433,...,95.0,64383208,1,a12924227433,}~biHmseN`Cg@bA]LA`@Lr@B`@Fd@B^FT?~ANx@CdB@h@K...,2,88.7,20.0,477.0,311.0
4,2,Volume,10723.9,4222,4278,87.0,Run,Run,NaN,12907973319,...,114.0,64383208,1,a12907973319,}~biHoseNt@Kr@Wf@CPMn@UTFv@Fv@L`@Bl@L`A@r@Hp@C...,2,88.0,23.0,522.0,316.0


## Calculating Fitness & Freshness

In [21]:
import math

def calculate_fitness_fatigue(training_loads, 
                              lambda_fitness=math.exp(-1/42), 
                              lambda_fatigue=math.exp(-1/7), 
                              initial_fitness=0.0, 
                              initial_fatigue=0.0):
    """
    Calculate daily Fitness and Fatigue values from a sequence of Training Loads.
    
    Parameters:
        training_loads (list or array-like): Daily training load values.
        lambda_fitness (float): Decay factor for Fitness.
        lambda_fatigue (float): Decay factor for Fatigue.
        initial_fitness (float): Starting Fitness value.
        initial_fatigue (float): Starting Fatigue value.
    
    Returns:
        fitness_values (list): Daily Fitness values.
        fatigue_values (list): Daily Fatigue values.
    """
    fitness_values = []
    fatigue_values = []
    
    F_fitness = initial_fitness
    F_fatigue = initial_fatigue
    
    for load in training_loads:
        # Update Fitness
        F_fitness = lambda_fitness * F_fitness + (1 - lambda_fitness) * load
        fitness_values.append(round(F_fitness))
        
        # Update Fatigue
        F_fatigue = lambda_fatigue * F_fatigue + (1 - lambda_fatigue) * load
        fatigue_values.append(round(F_fatigue))
    
    return fitness_values, fatigue_values


# Example usage:
training_loads_example = [7, 122, 10, 0, 148, 0, 123]  # Example: a week of loads
fitness_vals, fatigue_vals = calculate_fitness_fatigue(training_loads_example, initial_fitness=67, initial_fatigue=53)

print("Daily Fitness:", fitness_vals)
print("Daily Fatigue:", fatigue_vals)


Daily Fitness: [66, 67, 66, 64, 66, 64, 66]
Daily Fatigue: [47, 57, 51, 44, 58, 50, 60]


## Formula looks good, but it uses Training Impulse and not Training Load/Relative Effort (Training Impulse is found here : https://www.strava.com/athlete/fitness )

## From what I get, Training Impulse is Training Load/Suffer Score * 1.3, which pretty much matches what I see ? 

## Try and understand the "form" metric after ? 
Sources : 
https://www.reddit.com/r/Strava/comments/6f22g7/can_anyone_simply_explain_the_training_impulse/
https://fellrnr.com/wiki/TRIMP
https://medium.com/strava-engineering/fitness-freshness-updates-5d1057d67fac
https://www.trainingimpulse.com/banisters-trimp-0